In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('data/train.csv',parse_dates='datetime',index_col='datetime')
test = pd.read_csv('data/test.csv',parse_dates='datetime',index_col='datetime')

In [3]:
train[:20]

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
datetime,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032,0,1,1
2011-01-01 06:00:00,1,0,0,1,9.02,13.635,80,0.0000,2,0,2
2011-01-01 07:00:00,1,0,0,1,8.20,12.880,86,0.0000,1,2,3
2011-01-01 08:00:00,1,0,0,1,9.84,14.395,75,0.0000,1,7,8


In [4]:
train['hour'] = train.index.hour
train['month'] = train.index.month
train['year'] = train.index.year - 2011
train['day_of_week'] = train.index.weekday
train[:3]
test['hour'] = test.index.hour
test['month'] = test.index.month
test['year'] = test.index.year - 2011
test['day_of_week'] = test.index.weekday

In [5]:
from sklearn.linear_model import Lasso

In [27]:
clf = Lasso()

In [28]:
clf.fit(train['atemp'],train['count']);

In [29]:
prediction = clf.predict(train['count'])

In [30]:
print prediction

191.574131913


In [52]:
import numpy as np
def error(predictions,actual):
    print predictions[:10]
    print np.array(actual)[:10]
    err = 0
    for i, pred in enumerate(predictions):
        err += (np.log1p(pred)-np.log1p(actual[i]))**2
    J = (err/len(predictions))**.5
    return J

err=0
for i in train['count']:
    err += (np.log1p(prediction)-np.log1p(i))**2

print (err/1)**.5
#print error(prediction, train['count'])

163.723867294


In [53]:
import numpy as np
humidity = Lasso()
humidity.fit(train['humidity'],train['count'])
hPredict = humidity.predict(train['count'])
err=0
for i in train['count']:
    err += (np.log1p(hPredict)-np.log1p(i))**2

print (err/1)**.5

163.723867294


In [56]:
import numpy as np
twoFeats = Lasso()
features = ['humidity', 'atemp']
twoFeats.fit(train[features],train['count'])
predict = twoFeats.predict(train['count'])

print error(predict)

ValueError: shapes (10886,) and (2,) not aligned: 10886 (dim 0) != 2 (dim 0)